In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Rouge Score - Rouge(Recall-Oriented Understudy for Gisting Evaluation)

https://velog.io/@jochedda/Rouge-Score-Text-Summarization%EC%9D%98-%ED%8F%89%EA%B0%80%EC%A7%80%ED%91%9C


- 텍스트 요약 모델 성능 평가에 주로 사용되는 지표로 Label(사람이 만든 요약문)과 Summary(모델이 생성한 inference)를 비교해서 성능 계산

- ROUGE - N , ROUGE - L, ROUGE - W, ROUGE - S 등 다양한 지표가 존재함
각각 지표 별로 recall 및 precision을 둘 다 구하는 것이 좋다

- Recall
: label을 구성하는 단어 중 몇개가 inference 와 겹치는지 확인

    - 우선적으로 필요한 정보들이 다 담겨있는지 확인 !
- Precision
: inference를 구성하는 단어 중 몇개가 Label과 겹치는지 확인

    - 요약된 문장에 필요한 정보만을 얼마나 담고 있는지 확인 !

# ROUGE - N
- N-gram의 개수가 기준
- ROUGE - 1은 unigram, ROUGE - 2 는 bigram,…
- Recall: ouput과 겹치는 N-gram의 수 / Label의 N - gram의 수
- Precision : Label 과 겹치는 N-gram의 수 / output의 N-gram의 수

# ROUGE - L
- LCS(Longest common Sequence) between model output
- common sequence 중에서 가장 긴 것을 매칭
- N- gram과 달리 순서나 위치 관계를 고려한 알고리즘
- Recall : LCS 길이 / Label의 N-gram 수
- Precision : LCS 길이 / output의 N-gram 수

# ROUGE - S
- Skip - bigram을 활용한 metric
- 두 개의 토큰을 한 상으로 묵어서 ROUGE Score 계산
- 예를 들어, ‘ the brown fox’ 는 (the,brown),(brown,fox),(the,fox) 로 매핑되어 계산
- Window Size가 주어졌을 때, 이 사이즈 내에 위치하는 단어쌍들을 묶어 해당 단어쌍들이 얼마나 중복되는지 측정

# ROUGE Score의 단점
동음이의어에 대해서 평가할 수 없음 ! 즉, 같은 의미의 다른 단어가 많으면 성능을 낮게 측정한다.

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import random
import os

!pip install rouge
from rouge import Rouge


In [3]:
model_base = pd.read_csv('/content/drive/MyDrive/new_project/project2/test_result/base_model/result1_1.csv')
model_base.shape

(1000, 3)

In [4]:
model_base.head()

,text,origin_summary,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,대한민국 5G 홍보대사를 자처한 문재인 대통령은 5G 시대가 막 시작됐다는 기대감을...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"5세대 이동통신 서비스 '코리안 5G 테크-콘서트'가 열렸으며, 300여 명이 5G..."
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,박원순 서울시장은 '골목길 재생 시민 정책 대화'에서 서울시의 미래와 행복을 보장해...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,KB자산운용은 SK와 미국 알파벳의 간결한 지배구조를 배워라.


In [5]:
model1_10000 = pd.read_csv('/content/drive/MyDrive/new_project/project2/test_result/model1_10000/result1_1.csv')
model1_10000.shape

(1000, 3)

In [6]:
model1_10000.head()

,text,origin_summary,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,문재인 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 5G가 ...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,8일 서울 올림픽공원 K아트홀에서 5G 이동통신 서비스를 상용화한 것을 기념하는 '...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석해 ...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,KB자산운용은 기업 지배구조 개선 등을 통해 높은 수익률을 올리는 것을 목표로 하는...


In [7]:
def summary_len(model):
    len_total = 0
    for text in model['summary']:
        len_total += len(text)

    len_mean = len_total / len(model)
    return len_mean

In [8]:
## valid 요약본 길이

origin_len_total = 0
for text in model_base['origin_summary']:
    origin_len_total += len(text)

origin_len = origin_len_total / len(model_base)
print('valid 요약본 길이', origin_len)

## 모델 통과한 요약본 길이

model_list = ['model_base', 'model1_10000']

for model in model_list:
    temp = globals()[model]
    print(f'{model} 요약본 길이 : ', summary_len(temp))

valid 요약본 길이 133.457
model_base 요약본 길이 :  71.612
model1_10000 요약본 길이 :  113.838


In [16]:
rouge = Rouge()

test_result = []

for model in model_list:
    temp = globals()[model]

    rogue1_recall_total = 0
    rogue1_precision_total = 0
    rogue1_f1_total = 0

    rogue2_recall_total = 0
    rogue2_precision_total = 0
    rogue2_f1_total = 0

    rogueL_recall_total = 0
    rogueL_precision_total = 0
    rogueL_f1_total = 0

    for i in tqdm(range(len(temp))):
        origin_summary = temp['origin_summary'][i]
        model_summary = temp['summary'][i]

        test_score = rouge.get_scores(model_summary, origin_summary, avg=True)

        rogue1_recall_total += test_score['rouge-1']['r']
        rogue1_precision_total += test_score['rouge-1']['p']
        rogue1_f1_total += test_score['rouge-1']['f']

        rogue2_recall_total += test_score['rouge-2']['r']
        rogue2_precision_total += test_score['rouge-2']['p']
        rogue2_f1_total += test_score['rouge-2']['f']

        rogueL_recall_total += test_score['rouge-l']['r']
        rogueL_precision_total += test_score['rouge-l']['p']
        rogueL_f1_total += test_score['rouge-l']['f']

    rogue1_recall = rogue1_recall_total / len(temp)
    rogue1_precision = rogue1_precision_total / len(temp)
    rogue1_f1 = rogue1_f1_total / len(temp)

    rogue2_recall = rogue2_recall_total / len(temp)
    rogue2_precision = rogue2_precision_total / len(temp)
    rogue2_f1 = rogue2_f1_total / len(temp)

    rogueL_recall = rogueL_recall_total / len(temp)
    rogueL_precision = rogueL_precision_total / len(temp)
    rogueL_f1 = rogueL_f1_total / len(temp)

    test_result.append([model, rogue1_recall, rogue1_precision, rogue1_f1, rogue2_recall, rogue2_precision, rogue2_f1, rogueL_recall, rogueL_precision, rogueL_f1])




  0%|          | 0/1000 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not str

In [15]:
df_results = pd.DataFrame(test_result)
df_results.columns = ['model', 'rouge1_recall', 'rouge1_precision', 'rouge1_f1', 'rouge2_recall', 'rouge2_precision', 'rouge2_f1', 'rougeL_recall', 'rougeL_precision', 'rougeL_f1']
df_results

,model,rouge1_recall,rouge1_precision,rouge1_f1,rouge2_recall,rouge2_precision,rouge2_f1,rougeL_recall,rougeL_precision,rougeL_f1
0,model_base,0.128001,0.284671,0.169801,0.054420,0.131720,0.073937,0.118493,0.264885,0.157497
1,model1_10000,0.196048,0.325324,0.234023,0.098931,0.176694,0.120892,0.182393,0.303687,0.217971


In [17]:
model_out = ["he began by starting a five person war cabinet and included chamberlain as lord president of the council",
             "the siege lasted from 250 to 241 bc, the romans laid siege to lilybaeum",
             "the original ocean water was found in aquaculture"]

reference = ["he began his premiership by forming a five-man war cabinet which included chamberlain as lord president of the council",
             "the siege of lilybaeum lasted from 250 to 241 bc, as the roman army laid siege to the carthaginian-held sicilian city of lilybaeum",
             "the original mission was for research into the uses of deep ocean water in ocean thermal energy conversion (otec) renewable energy production and in aquaculture"]
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

[{'rouge-1': {'r': 0.7368421052631579,
   'p': 0.7777777777777778,
   'f': 0.7567567517604091},
  'rouge-2': {'r': 0.5, 'p': 0.5294117647058824, 'f': 0.514285709289796},
  'rouge-l': {'r': 0.7368421052631579,
   'p': 0.7777777777777778,
   'f': 0.7567567517604091}},
 {'rouge-1': {'r': 0.5882352941176471,
   'p': 0.9090909090909091,
   'f': 0.7142857095153062},
  'rouge-2': {'r': 0.38095238095238093,
   'p': 0.6153846153846154,
   'f': 0.4705882305709343},
  'rouge-l': {'r': 0.5882352941176471,
   'p': 0.9090909090909091,
   'f': 0.7142857095153062}},
 {'rouge-1': {'r': 0.3333333333333333, 'p': 0.875, 'f': 0.48275861669441145},
  'rouge-2': {'r': 0.125, 'p': 0.42857142857142855, 'f': 0.19354838360041626},
  'rouge-l': {'r': 0.2857142857142857, 'p': 0.75, 'f': 0.4137930994530322}}]